In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 


def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))



customer_id = '144'
customer_name='%motionapac%'
client='Motion APAC'

In [2]:
dateszs='2001-08-11'
attribut='features'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
1336933


# Check for Attribute bucket Values

In [3]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')

# lst_attribute=['color']
for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass

    
    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

140
Number of attributes: 140

Start Buckets
0 Attribute: amperage
Number of SKUs for the attribute amperage: 64
Start Buckets
1 Attribute: battery_size
Number of SKUs for the attribute battery_size: 47
Start Buckets
2 Attribute: bearing_cap_diameter
Number of SKUs for the attribute bearing_cap_diameter: 0
Start Buckets
3 Attribute: bearing_cap_diameter_converted
Number of SKUs for the attribute bearing_cap_diameter_converted: 0
Start Buckets
4 Attribute: bed_size
Number of SKUs for the attribute bed_size: 1
Start Buckets
5 Attribute: belt_type
Number of SKUs for the attribute belt_type: 18449
Start Buckets
6 Attribute: beta_ratio
Number of SKUs for the attribute beta_ratio: 0
Start Buckets
7 Attribute: blade_count
Number of SKUs for the attribute blade_count: 0
Start Buckets
8 Attribute: blade_diameter
Number of SKUs for the attribute blade_diameter: 0
Start Buckets
9 Attribute: blade_length
Number of SKUs for the attribute blade_length: 485
Start Buckets
10 Attribute: blade_material


# Check Freetext Attributes

In [ ]:

btu=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+|\,\d+)?(?!\,) BTU)|(n\/a)|()'''
can=r'''((?<!\d)(?<!\,)\d+\-can)|(n\/a)|()'''
cubic_feet=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+)?(?!\,) cu ft)|(n\/a)|()'''
cup=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,)\-Cup)|(n\/a)|()'''

dots_per_inch_number_X=r'''((?<!\,)(?<!\d)(?<!X )\d+(?!\,)(?: x \d+)? DPI)|(n\/a)|()'''
dynamic_load_capacity=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+)?(?!\,) (?:lb))|(n\/a)|()'''

feet_thousand_seperator=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+(?:\.\d+)?|\.\d+)?(?!\,) (?:ft|in))|(n\/a)|()'''
feet_mm=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+(?:\.\d+)?|\.\d+)?(?!\,) (?:m|in))|(n\/a)|()'''
feet_mi=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:ft|mi))|(n\/a)|()'''
flow_rate_GPM=r'''(\d+ GPM)|(n\/a)|()'''
flow_rate_GPM=r'''(\d+ GPM)|(n\/a)|()'''
frequency_rate_hz=r'''((?<!\,)(?<!\d)\d+(?:\,\d+)? (?:Hz))|(n\/a)|()'''
g_force=r'''(\d+G)|(n\/a)|()'''
gauge=r'''(\d+ Gauge)|(n\/a)|()'''
gb_tb_mb=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+)?(?!\,)(?:TB|GB|MB))|(n\/a)|()'''
ghz=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:GHz))|(n\/a)|()'''


horsepower=r'(\d+ HP)|(n\/a)|()'
hour=r'(\d+(?:\.\d+)? hr)|(n\/a)|()'
hours=r'((?<!\d)(?<!\,)\d+(?:\,\d+)?(?!\,)(?!\d) hrs)|(n\/a)|()'
hz=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:Hz))|(n\/a)|()'''
inch=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) in)|(n\/a)|()'''
inch_feet=r'((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+|\,\d+)?(?!\,) (?:in|ft|m))|(n\/a)|()'
inch_fraction=r'''(\d+(?:(?:\.\d+)?|\d+\-\d+\/\d+|(?: |\-)\d+\/\d+|\/\d+) in)|(n\/a)|()'''
inch_mm=r'''((?<!\d)(?<!\,)\d+(?:\-\d+\/\d+|\/\d+)?(?!\,)(?!\d) (?:in|mm))|(n\/a)|()'''


mega_pixels=r'''(\d+ MP)|(n\/a)|()'''
mg=r'(\d+ mg)|(n\/a)|()'
micron=r'''(\d+ micron)|(n\/a)|()'''
mil=r'''(\d+(?:\.\d+)? mil)|(n\/a)|()'''
milliamp_hours=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+)?(?!\,)(?:mAh))|(n\/a)|()'''
minute=r'''((?<!\d)(?<!\,)\d+ min)|(n\/a)|()'''
mm=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) mm)|(n\/a)|()'''
ms=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:ms))|(n\/a)|()'''
nit=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) nit)|(n\/a)|()'''
number_space_fraction=r'''(\d+(?:(?:\.\d+)?|\d+\-\d+\/\d+| \d+\/\d+) in)|(n\/a)|()'''
number_with_sperator=r'''((?<!\d)\d+(?:\,\d+|\d\d)?(?!\,)(?!\d))|(n\/a)|()'''
number_no_seperator=r'''((?<!\d)(?<!\,)\d+(?!\,)(?!\d))|(n\/a)|()'''


page=r'''(\d+ pages)|(n\/a)|()'''
peak_transfer_speed=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:Gbps|Mbps|Kbps|MBps|GBps))|(n\/a)|()'''
person=r'''(1 person)|([2-9] people)|(\d\d+ people)|(n\/a)|()'''
pieces=r'''(1 Piece)|((?:\d\d+|[2-9]) Pieces)|(n\/a)|()'''
pound=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+|\,\d+)?(?!\,) lb)|(n\/a)|()'''
psi=r'(\d+ PSI)|(n\/a)|()'


ram=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,)(?:GB|MB))|(n\/a)|()'''
rpm=r'((?<!\d)(?<!\,)\d+(?:\,\d+)?(?!\,)(?!\d) RPM)|(n\/a)|()'
sec=r'(\d+(?:\.\d+)? sec)|(n\/a)|()'
square_feet=r'(\d+(?:\,\d+)? sq ft)|(n\/a)|()'
tpi=r'''(\d+ tpi)|(n\/a)|()'''
temperature=r'''(\d+ F)|(n\/a)|()'''
volt=r'''(\d+V)|(n\/a)|()'''
wat=r'''(\d+W)|(n\/a)|()'''
year=r'((?:[2-9]|\d\d+) years)|(1 year)|(\d+ months)|(n\/a)|()'
# awg=r'''(\d+AWG)|(n\/a)|()'''










amp=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+)?(?!\,) A)|(n\/a)|()'''
capacity=r'''((?<!\,)(?<!\d)(?<!\-)(?<!X )\d+(?:\.\d+|\/\d+|\-\d+\/\d+|\,\d+)?(?!\,) (?:ml|L|lb|gal|kg|g|mm|in|oz))|(n\/a)|()'''
classs=r'''((?:A))|(n\/a)|()'''
db=r'((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) (?:dB))|(n\/a)|()'
grit=r'''((?<!\d)(?<!\,)\d+(?: \- \d+)?(?!\,)(?!\d))|(n\/a)|()'''
horsepower=r'(\d+ HP)|(n\/a)|()'
inch_mm=r'''((?<!\,)(?<!\d)(?<!\.)(?<!0)(?<!\/)(?<!\-)(?<!X )\d+(?:\.\d+|\/\d+|\-\d+\/\d+| \d+\/\d+)?(?!\,)(?!\.) (?:mm|in|m))|(n\/a)|()'''
lumens=r'''(\d+ lumens)|(n\/a)|()'''
number_no_seperator=r'''((?<!\d)(?<!\,)\d+(?!\,)(?!\d))|(n\/a)|()'''

patterns = {
    'amperage': amp,
    'bearing_cap_diameter': inch_mm,
    'blade_count': number_no_seperator,
    'blade_diameter': inch_mm,
    'blade_length': inch_mm,
    'blade_width': inch_mm,
    'bolt_length'inch_mm: ,
    'bore_diameter': inch_mm,
    'capacity': capacity,
    'chain_size': inch_mm,
    'class': classs,
    'compressed_length': inch_mm,
    'cone_width': inch_mm,
    'count': number_no_seperator,
    'cup_width': inch_mm,
    'cutting_depth': inch_mm,
    'decibels': db,
    'diameter': inch_mm,
    'drawer_count': number_no_seperator,
    'drive_size': inch_mm,
    'gauge': number_no_seperator,
    'grit': grit,
    'hex_size': inch_mm,
    'hole_diameter': inch_mm,
    'horsepower': horsepower,
    'inside_diameter': inch_mm,
    'length_through_bore': inch_mm,
    'lumens': lumens,
    'maximum_flow_rate': ,
    'maximum_pressure': ,
    'maximum_psi': ,
    'maximum_rpm': ,
    'maximum_runtime': ,
    'noise_reduction_rating_nrr': ,
    'nominal_bore': ,
    'open_width': ,
    'outside_diameter': ,
    'overall_depth': ,
    'overall_height': ,
    'overall_length': ,
    'overall_width': ,
    'pin_diameter': ,
    'pipe_size': ,
    'pitch': ,
    'pitch_length': ,
    'pressure': ,
    'range': ,
    'rell_capacity': ,
    'shaft_diameter': ,
    'shank_thread_size': ,
    'sheet_count': ,
    'socket_size': ,
    'strokes_per_minute': ,
    'teeth': ,
    'teeth_per_inch': ,
    'thickness': ,
    'thread_per_inch': ,
    'thread_size': ,
    'tip_diameter': ,
    'torque': ,
    'travel_length': ,
    'voltage': ,
    'volume': ,
    'wattage': ,
    'weight': ,
    'weight_capacity': ,
    'weight_converted': 
}

In [25]:
# cb['attribute'].explode().value_counts()
cb[cb['attribute'].astype(str)=='maximum_flow_rate']['value'].explode().value_counts()[500:]

Series([], Name: value, dtype: int64)

In [ ]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

# Curated

# Uncurated

In [ ]:
# parameters
formatted_attribute = 'sustainability'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

# send to the folder for upload

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good(client, attribute,matchesna,today)   

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good(client, attribute,matchesdiam,today)  